# Module 3: JUMP Analysis 

In this notebook, we utilize the Joint Undertaking in Morphological Profile [dataset](https://jump-cellpainting.broadinstitute.org/cell-painting) and integrate it into our model. Our objective is to assess the probability of specific cell injuries present within each well entry from the JUMP dataset. 

Additionally, we identify shared treatments between the JUMP and cell-injury datasets to construct a confusion matrix. This enables us to evaluate the performance of predicting cellular injury across different datasets.

In [1]:
import sys
import pathlib

import joblib
import pandas as pd

# project module imports
sys.path.append("../../")  # noqa
from src.utils import (
    load_json_file,
    split_meta_and_features,
    check_feature_order,
    generate_confusion_matrix_tl,
)

## Setting up parameters and paths

In [2]:
# setting up paths and output paths
results_dir = pathlib.Path("../../results")
fs_results_dir = (results_dir / "0.feature_selection").resolve(strict=True)
data_split_dir = (results_dir / "1.data_splits/").resolve(strict=True)
jump_data_dir = pathlib.Path("../../data/JUMP_data").resolve(strict=True)
modeling_dir = pathlib.Path("../../results/2.modeling").resolve(strict=True)

# JUMP data files
jump_data_path = (jump_data_dir / "JUMP_all_plates_normalized_negcon.csv.gz").resolve(
    strict=True
)

# loading only cell injury metadata (after holdout has been applied)
cell_injury_metadata_path = (
    data_split_dir / "cell_injury_metadata_after_holdout.csv.gz"
).resolve(strict=True)

# model paths
multi_class_model_path = (modeling_dir / "multi_class_model.joblib").resolve(
    strict=True
)
shuffled_multi_class_model_path = (
    modeling_dir / "shuffled_multi_class_model.joblib"
).resolve(strict=True)

# overlapping feature space path
shared_feature_space_path = (
    fs_results_dir / "cell_injury_shared_feature_space.json"
).resolve(strict=True)

# injury codes
injury_codes_path = (fs_results_dir / "injury_codes.json").resolve(strict=True)

# output paths
jump_analysis_dir = (results_dir / "3.jump_analysis").resolve()
jump_analysis_dir.mkdir(exist_ok=True)

## Loading in datasets and json files

Here we are loading the JUMP dataset along with the cell injury metadata, injury codes and the files representing the overlapping feature space.

In [3]:
# loading in JUMP dataset
jump_df = pd.read_csv(jump_data_path)

# loading in cell injury metadata only (after holdout)
cell_injury_meta_df = pd.read_csv(cell_injury_metadata_path)

# split metadata and feature columns
jump_meta, jump_feats = split_meta_and_features(jump_df, metadata_tag=True)

# loading json file that contains the shared feature
injury_codes = load_json_file(injury_codes_path)
injury_encoder = injury_codes["encoder"]
injury_decoder = injury_codes["decoder"]

# loading in shared feature space
shared_feature_space = load_json_file(shared_feature_space_path)
shared_features = shared_feature_space["features"]

# Display data
print("JUMP dataset shape", jump_df.shape)
print("Number of Meta features", len(jump_meta))
print("Number of JUMP features", len(jump_feats))
print("Number of shared features between JUMP and Cell Injury", len(shared_features))
jump_df.head()

JUMP dataset shape (19498, 5805)
Number of Meta features 13
Number of JUMP features 5792
Number of shared features between JUMP and Cell Injury 221


,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,...,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256,Metadata_target_sequence,Metadata_negcon_control_type,Metadata_solvent,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,Metadata_smiles
0,ccsbBroad304_00900,BR00117006,A01,KCNN1,trt,NaN,-0.51950,-0.17888,-1.43320,-0.99033,...,0.093186,0.165130,0.129190,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ccsbBroad304_07795,BR00117006,A02,SLCO2B1,trt,NaN,0.72735,0.76863,-0.55811,-1.11400,...,0.464660,0.489040,0.454050,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ccsbBroad304_02826,BR00117006,A03,SLC7A11,trt,NaN,0.29546,0.33018,0.26425,-0.87030,...,-0.008241,0.000635,0.025713,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ccsbBroad304_01492,BR00117006,A04,SCNN1G,trt,NaN,0.52579,0.56077,-1.94300,-0.53439,...,-0.600320,-0.563980,-0.591390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ccsbBroad304_00691,BR00117006,A05,GRIN2A,trt,NaN,-0.58868,-0.82855,-0.31369,-0.91332,...,-1.335800,-1.330000,-1.328400,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Updating the JUMP Dataset by Selecting Only Shared Features

During this step, we utilize the shared feature list to update our JUMP dataset, selecting only those features that overlap.

Note that the shared feature space file maintains the same order as the feature space used during model training.

In [4]:
# update the over lapping jump df
# Augment the overlapping feature space with the metadata
shared_jump_df = jump_df[shared_features]
shared_jump_df = pd.concat([jump_df[jump_meta], shared_jump_df], axis=1)

# # split the features
shared_meta, shared_feats = split_meta_and_features(shared_jump_df, metadata_tag=True)

# # checking if the feature space are identical (also looks for feature space order)
assert check_feature_order(
    ref_feat_order=shared_features, input_feat_order=shared_feats
), "Feature space are not identical"

# display
print(
    "Shape of overlapping jump datadrame with overlapping features",
    shared_jump_df.shape,
)
print("Number of meta features", len(shared_meta))
print("Number of features", len(shared_feats))
shared_jump_df.head()

Shape of overlapping jump datadrame with overlapping features (19498, 234)
Number of meta features 13
Number of features 221


,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,Metadata_solvent,Metadata_InChIKey,...,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_RadialDistribution_MeanFrac_ER_3of4,Cells_AreaShape_Center_Y,Nuclei_AreaShape_Zernike_9_3,Cells_RadialDistribution_FracAtD_AGP_3of4,Cytoplasm_RadialDistribution_MeanFrac_DNA_2of4,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_5_5,Cells_RadialDistribution_MeanFrac_DNA_4of4,Nuclei_Intensity_IntegratedIntensityEdge_RNA
0,ccsbBroad304_00900,BR00117006,A01,KCNN1,trt,NaN,NaN,NaN,NaN,NaN,...,-0.98742,0.43815,-0.99809,-0.010594,1.627800,0.26845,-0.060704,-1.023200,-1.24510,0.761840
1,ccsbBroad304_07795,BR00117006,A02,SLCO2B1,trt,NaN,NaN,NaN,NaN,NaN,...,0.38401,-1.13920,-1.14100,-2.015200,0.433890,-0.10145,0.183240,0.192440,-1.03640,1.739800
2,ccsbBroad304_02826,BR00117006,A03,SLC7A11,trt,NaN,NaN,NaN,NaN,NaN,...,0.27789,-1.61190,-0.87242,-0.519110,0.405700,-0.38859,0.109040,0.299140,-0.65195,0.794260
3,ccsbBroad304_01492,BR00117006,A04,SCNN1G,trt,NaN,NaN,NaN,NaN,NaN,...,0.54864,-0.73557,-0.58255,-0.962890,-0.065436,-0.13684,-0.002248,0.070498,-0.83026,0.088251
4,ccsbBroad304_00691,BR00117006,A05,GRIN2A,trt,NaN,NaN,NaN,NaN,NaN,...,-0.37075,-1.69150,-0.84954,-0.314290,0.447980,-1.69020,-0.589060,-0.184820,0.28520,-0.850090


In [5]:
# save overlapping files
shared_jump_df.to_csv(
    jump_analysis_dir / "shared_feats_jump_data.csv.gz",
    compression="gzip",
    index=False,
)

## Identifying shared treatments
Once the feature space has been narrowed down to only those features shared between both datasets, the next step is to generate a dataset containing shared treatments that are both presentin in the `cell_injury` and `JUMP` datasets. These shared compounds will then be utilized to subset the `JUMP` dataset, which will be considered as the ground truth for downstream analyses.

**Approach**:
1. **Identifying shared Compounds**: We compare the compounds present in both datasets to identify the overlapping ones.
2. **Subsetting the JUMP Dataset**: Once the overlapping compounds are identified, we subset the `JUMP` dataset to include only those compounds, forming our ground truth dataset.
3. **Save dataset**: The dataset will be saved in the `./results/3.jump_analysis`

### Identifying Overlapping Compounds
Here, we used the International Chemical Identifier (InChI) to identify chemicals shared between the JUMP dataset and the Cell Injury dataset.

In [6]:
cell_injury_InChI_keys = cell_injury_meta_df["Compound InChIKey"].unique().tolist()
jump_InChI_keys = shared_jump_df["Metadata_InChIKey"].unique().tolist()

# # identify common InChI Keys
common_compounds_inchikey = list(
    set(cell_injury_InChI_keys).intersection(jump_InChI_keys)
)

# # identify the compounds and display in cell injury data
overlapping_compounds_df = cell_injury_meta_df.loc[
    cell_injury_meta_df["Compound InChIKey"].isin(common_compounds_inchikey)
]
unique_compound_names = overlapping_compounds_df["Compound Name"].unique().tolist()
print("Identified overlapping compounds:", ", ".join(unique_compound_names))

# now create a dataframe where it contains the injury code, name and injury type
overlapping_compounds_df = (
    overlapping_compounds_df[
        ["injury_code", "injury_type", "Compound Name", "Compound InChIKey"]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# lower casing all the entries
overlapping_compounds_df["injury_type"] = overlapping_compounds_df[
    "injury_type"
].str.lower()
overlapping_compounds_df

Identified overlapping compounds: DMSO, Colchicine, Cycloheximide, Menadione


,injury_code,injury_type,Compound Name,Compound InChIKey
0,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N
1,1,cytoskeletal,Colchicine,IAKHMKGGTNLKSZ-INIZCTEOSA-N
2,5,miscellaneous,Cycloheximide,YPHMISFOHDHNIV-FSZOTQKASA-N
3,6,redox,Menadione,MJVAVZPDRWSRRC-UHFFFAOYSA-N


Once the common compounds and their associated cell injury types are identified, the next step involves selecting it from the JUMP dataset to select only wells that possess the common InChI keys.

In [7]:
# selecting rows that contains the overlapping compounds
shared_treat_jump_df = shared_jump_df.loc[
    shared_jump_df["Metadata_InChIKey"].isin(common_compounds_inchikey)
]

# augment filtered JUMP data with labels
shared_treat_jump_df = pd.merge(
    overlapping_compounds_df,
    shared_treat_jump_df,
    right_on="Metadata_InChIKey",
    left_on="Compound InChIKey",
)

# shared treatment jump df
print("shape: ", shared_treat_jump_df.shape)
shared_treat_jump_df.head()

shape:  (1593, 238)


,injury_code,injury_type,Compound Name,Compound InChIKey,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,...,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_RadialDistribution_MeanFrac_ER_3of4,Cells_AreaShape_Center_Y,Nuclei_AreaShape_Zernike_9_3,Cells_RadialDistribution_FracAtD_AGP_3of4,Cytoplasm_RadialDistribution_MeanFrac_DNA_2of4,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_5_5,Cells_RadialDistribution_MeanFrac_DNA_4of4,Nuclei_Intensity_IntegratedIntensityEdge_RNA
0,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00116991,A02,NaN,control,negcon,...,-0.30877,-3.756800,-1.7252,-0.72192,0.32788,-1.86520,-0.39931,-0.22825,-0.68427,-3.17420
1,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00116991,A09,NaN,control,negcon,...,0.86030,-2.030700,-1.5569,-0.45467,-0.07307,-0.22281,-0.87502,-0.94121,-2.17130,-2.45920
2,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00116991,A17,NaN,control,negcon,...,1.95790,-0.166810,-1.0716,-2.24100,-0.83749,1.62220,1.90980,-0.38213,-1.11500,-0.69815
3,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00116991,B03,NaN,control,negcon,...,-1.57290,-0.674490,-1.5162,-0.12379,-0.26792,-0.60054,-0.36980,-0.24877,1.31050,0.26369
4,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00116991,B14,NaN,control,negcon,...,0.49182,-0.065273,0.1620,-0.38179,-0.54147,0.22766,1.29000,-1.55670,0.27310,-0.47791


Now that we have identified the wells treated with overlapping treatments, we want to know the amount of wells that a specific treatment have.

In [8]:
# count number of wells and agument with injury_code injury_yype and compound name
well_counts_df = (
    shared_treat_jump_df.groupby("Metadata_InChIKey")
    # counting the numbver of wells
    .size()
    .to_frame()
    .reset_index()
    # merge based on InChIKey
    .merge(
        overlapping_compounds_df,
        left_on="Metadata_InChIKey",
        right_on="Compound InChIKey",
    )
    # remove duplicate InChIKey Column
    .drop(columns=["Compound InChIKey"])
)

# update columns
well_counts_df.columns = [
    "Metadata_InChIKey",
    "n_wells",
    "injury_code",
    "injury_type",
    "compund_name",
]
well_counts_df

,Metadata_InChIKey,n_wells,injury_code,injury_type,compund_name
0,IAKHMKGGTNLKSZ-INIZCTEOSA-N,24,1,cytoskeletal,Colchicine
1,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,1522,0,control,DMSO
2,MJVAVZPDRWSRRC-UHFFFAOYSA-N,23,6,redox,Menadione
3,YPHMISFOHDHNIV-FSZOTQKASA-N,24,5,miscellaneous,Cycloheximide


Finally we save the shared_treaments_df as a csv.gz file.

In [9]:
# save overlapping files
shared_treat_jump_df.to_csv(
    jump_analysis_dir / "shared_treatments_jump_data.csv.gz",
    compression="gzip",
    index=False,
)

## Applying JUMP dataset to Multi-Class Logistics Regression Model

In [10]:
# split the data
aligned_meta_cols, aligned_feature_cols = split_meta_and_features(shared_jump_df)

# check if the feature space are the same
X = shared_jump_df[aligned_feature_cols]


assert check_feature_order(
    ref_feat_order=shared_features, input_feat_order=X.columns.tolist()
), "Feature space are not identical"

In [11]:
# Loading in model
model = joblib.load(modeling_dir / "multi_class_model.joblib")
shuffled_model = joblib.load(modeling_dir / "shuffled_multi_class_model.joblib")

Here, we apply the JUMP dataset to the model to calculate the probabilities of each injury being present per well. These probabilities are then saved in a tidy long format suitable for plotting in R.

In [16]:
# get all injury classes
injury_classes = [injury_decoder[str(code)] for code in model.classes_.tolist()]

# prediction probabilities on both non-shuffled and shuffled models
y_proba = model.predict_proba(X)
shuffled_y_proba = shuffled_model.predict_proba(X)

# convert to pandas dataframe
y_proba_df = pd.DataFrame(y_proba)
shuffled_y_proba_df = pd.DataFrame(shuffled_y_proba)

# # update column names with injury type names
y_proba_df.columns = [
    injury_codes["decoder"][str(colname)] for colname in y_proba_df.columns.tolist()
]

shuffled_y_proba_df.columns = [
    injury_codes["decoder"][str(colname)]
    for colname in shuffled_y_proba_df.columns.tolist()
]

# # # adding column if labels indicating if the prediction was done with a shuffled model
y_proba_df.insert(0, "shuffled_model", False)
shuffled_y_proba_df.insert(0, "shuffled_model", True)


# # # concat all probabilities into one dataframe
all_probas_df = pd.concat([y_proba_df, shuffled_y_proba_df]).reset_index(drop=True)

# # # Add a column to indicate the most probable injury
# # This is achieved by selecting the injury with the highest probability
all_probas_df.insert(
    0,
    "pred_injury",
    all_probas_df[injury_classes].apply(lambda row: row.idxmax(), axis=1),
)

# # # next is to convert the probabilities dataframe into tidy long
all_probas_df_tl = pd.melt(
    all_probas_df,
    id_vars=["shuffled_model", "pred_injury"],
    value_vars=injury_classes,
    var_name="injury_type",
    value_name="proba",
)

# # save probabilities in tidy long format
# all_probas_df_tl.to_csv(jump_analysis_dir / "JUMP_injury_proba.csv.gz", index=False)

# print("tidy long format probability shape", all_probas_df_tl.shape)

In [17]:
all_probas_df

,pred_injury,shuffled_model,Control,Cytoskeletal,Hsp90,Kinase,Genotoxin,Miscellaneous,Redox,HDAC,mTOR,Proteasome,Saponin,Mitochondria,Ferroptosis,Tannin,Nonspecific reactive
0,Ferroptosis,False,1.438464e-17,4.131411e-16,2.955995e-09,1.027191e-11,1.057821e-10,1.632949e-13,8.903193e-10,1.406394e-11,1.116487e-17,1.453965e-11,5.714721e-14,9.895299e-08,9.999999e-01,6.826247e-15,3.554143e-15
1,Mitochondria,False,1.715604e-12,5.142734e-13,4.551879e-11,7.879606e-11,1.771835e-09,4.129903e-10,2.927905e-10,5.332475e-09,2.609426e-15,2.247679e-10,9.902593e-14,1.000000e+00,3.701192e-10,1.087774e-09,4.356712e-12
2,Mitochondria,False,1.297985e-09,5.021454e-10,2.134424e-11,1.867954e-08,6.890035e-07,2.888854e-07,5.023377e-07,1.692330e-04,5.171636e-10,2.922721e-09,1.142617e-10,9.998293e-01,1.490886e-09,2.526689e-09,2.190264e-10
3,Mitochondria,False,8.163913e-14,2.032184e-16,2.537253e-14,2.302486e-09,1.430685e-13,2.299701e-11,2.175275e-10,1.648090e-16,1.896917e-11,8.339973e-16,8.653778e-17,9.999999e-01,2.049222e-09,7.025376e-15,1.365756e-07
4,Mitochondria,False,8.853317e-08,2.847344e-10,7.610846e-10,1.553992e-04,1.688966e-08,4.466594e-05,1.065114e-08,3.606669e-07,1.477434e-05,2.231413e-11,4.148914e-15,9.994746e-01,5.824651e-11,1.376133e-10,3.101235e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38991,Ferroptosis,True,5.387225e-17,1.288473e-17,1.659498e-16,4.636982e-18,8.638888e-18,3.234542e-17,3.472438e-17,7.371799e-10,5.477744e-15,2.538219e-24,2.755151e-23,6.040665e-17,1.000000e+00,4.929165e-23,5.600868e-21
38992,Ferroptosis,True,9.619139e-10,3.747176e-10,2.259675e-08,5.747855e-10,3.195269e-10,5.069327e-10,7.297097e-11,1.742041e-05,6.092124e-09,1.145976e-13,1.280626e-10,1.613291e-12,9.999825e-01,2.968223e-10,8.060363e-15
38993,Ferroptosis,True,7.544787e-06,3.963694e-06,3.407060e-05,1.511341e-06,1.696112e-06,1.824880e-06,2.310011e-05,1.310931e-01,3.553815e-04,6.885886e-13,4.861152e-09,2.736368e-09,8.684778e-01,5.751648e-08,3.678072e-10
38994,Ferroptosis,True,3.185143e-14,5.528401e-15,5.265937e-15,5.189713e-15,4.746876e-15,1.670657e-14,6.065998e-13,1.650469e-07,3.412141e-13,1.447967e-19,3.871946e-22,6.477836e-14,9.999998e-01,3.589833e-21,3.829825e-16


In [28]:
all_probas_df_tl.iloc[:15]

,shuffled_model,pred_injury,injury_type,proba
0,False,Ferroptosis,Control,1.438464e-17
1,False,Mitochondria,Control,1.715604e-12
2,False,Mitochondria,Control,1.297985e-09
3,False,Mitochondria,Control,8.163913e-14
4,False,Mitochondria,Control,8.853317e-08
5,False,Mitochondria,Control,1.646288e-10
6,False,Mitochondria,Control,8.155046e-12
7,False,Mitochondria,Control,3.726066e-13
8,False,Mitochondria,Control,3.760556e-11
9,False,Mitochondria,Control,8.580785e-07


## Generating Confusion Matrix

In [13]:
shared_treat_meta, shared_treat_feats = split_meta_and_features(shared_treat_jump_df)
shared_X = shared_treat_jump_df[shared_treat_feats]
shared_y = shared_treat_jump_df["injury_code"]

In [14]:
jump_overlap_cm = generate_confusion_matrix_tl(
    model, shared_X, shared_y, shuffled=False, dataset_type="JUMP Overlap"
).fillna(0)
shuffled_jump_overlap_cm = generate_confusion_matrix_tl(
    shuffled_model, shared_X, shared_y, shuffled=True, dataset_type="JUMP Overlap"
).fillna(0)

In [15]:
# save confusion matrix
pd.concat([jump_overlap_cm, shuffled_jump_overlap_cm]).to_csv(
    modeling_dir / "jump_overlap_confusion_matrix.csv.gz",
    compression="gzip",
    index=False,
)